In [1]:
import numpy as np

In [2]:
def Mixtures(tauO, p_arms):
    p_out = 0
    for i in range(len(tauO)):
        p_out += tauO[i]*p_arms[i]
    return p_out

In [3]:
def Utility(p):
    #utility function
    return p*(1-p)

In [4]:
def ETC(K=2, T = 100000, pb = [0.35, 0.85], v=2):
    eps = (K/T)**(1/2)*np.sqrt(np.log(T))
    I = np.linspace(0, 1, int(np.ceil(1/eps)))
    
    MAX_U = 0 
    for alpha_bar in I:
        for i in range(K):
            for j in range(i+1, K):
                temp_alpha = np.zeros((K))
                temp_alpha[i] = alpha_bar
                temp_alpha[j] = 1-alpha_bar

                temp_U = Utility(Mixtures(temp_alpha, pb))
                
                if temp_U > MAX_U:
                    MAX_U = temp_U
                
                
                
    
    DELTA = 1/4 - MAX_U
    
    #N = K**(1+2/q)/v*(2*L/DELTA)**(2/q)*np.log(2*K*T*((1/eps)**(K-1)+1))
    #For the T values we have experimented, N > T and hence, to be able run experiments we have assigned a large value
    #to N instead which is T/2. For larger T values such that N < T, uncomment the line above.
    
    N = T/2
    

    tau_T = np.zeros((K))

    emp_pb = np.zeros((K))
    
    regrets = []
    
    opt_alpha = np.zeros((K))
    
    #EXPLORATION
    starttime = 0
    for i in range(K):
        for time in range(starttime, starttime + int(N/K)):
            reward = np.random.binomial(size=1, n=1, p=(pb[i]))
            
            emp_pb[i] = float(emp_pb[i]*tau_T[i] + reward)/(tau_T[i] + 1)
            
            tau_T[i] += 1
            
    
        starttime = time + 1
     
    
    max_UCB = 0 
    for alphaT in I:
        for i in range(K):
            for j in range(i+1, K):
                temp_alpha = np.zeros((K))
                temp_alpha[i] = alphaT
                temp_alpha[j] = 1-alphaT
                
                if Utility(Mixtures(temp_alpha, emp_pb)) > max_UCB:
                    max_UCB = Utility(Mixtures(temp_alpha, emp_pb))
                    opt_alpha = temp_alpha
                
  
    
    for i in range(K):
        chosen_arm = i
        stop_time = int(min(starttime + np.floor(T*opt_alpha)[i]-N/K, T))
    
        for time in range(starttime, stop_time):
            reward = np.random.binomial(size=1, n=1, p=(pb[chosen_arm]))

            emp_pb[chosen_arm] = float(emp_pb[chosen_arm]*tau_T[chosen_arm] + reward)/(tau_T[chosen_arm] + 1)

            tau_T[chosen_arm] += 1
        
        starttime = time + 1
        

        
    regret = 1/4-Utility(Mixtures(tau_T/T, pb))
        
    return regret, emp_pb, tau_T

    

## Time Horizon Experiments


T_list := list of different time horizons <br />
file_path := name of the txt file for data to be saved <br />
pb[0] := mean of the first arm <br />
pb[1] := mean of the second arm <br />

In [5]:
import os
T_list = [100000, 200000, 300000, 400000, 500000]
pb=[0.4, 0.9]

file_path = "Regrets_ETC_T.txt"
for T in T_list:
    for save_idx in range(1000):
        regret, emp_pb, tau_T = ETC(K=2, T = T, pb=pb, v=2)
        

        with open(file_path, "a+") as file:
            file.write(f'T={T}, K=2, p1={pb[0]}, p2={pb[1]}, RESULT: tau(T)={tau_T}, emp_pb={emp_pb} \n')

## Number of Arms Experiments


K_list := list of different number of arms <br />
file_path := name of the txt file for data to be saved <br />
p1 := mean of the first arm <br />
p2 := mean of the second arm <br />
zeta := exploration coefficient <br />

In [6]:
T = 300000
K_list = [2, 3, 4, 5, 6]
pb = [0.45, 0.60, 0.75, 0.9, 0.3 , 0.15]

file_path = "Regrets_ETC_K.txt"
for K in K_list:
    for save_idx in range(1000):
        regret, emp_pb, tau_T = ETC(K=K, T = T, pb=pb[:K], v=2)
        

        with open(file_path, "a+") as file:
            file.write(f'T={T}, K={K}, pb={pb}, RESULT: tau(T)={tau_T}, emp_pb={emp_pb} \n')
